In [1]:
import sys
sys.path.append('/Users/mike/Dropbox/science/_projects/')
sys.path.append('/Users/mike/github/QDMlab')
sys.path.append(r"C:\Users\micha\github\QDMlab")
sys.path.append(r"C:\Users\micha\Dropbox\science\_projects")

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
from pandas import ExcelWriter
import admin_tools
from admin_tools import arguments, docstring

In [3]:
code = admin_tools.preload_code()

In [4]:
def call(file):
    return ' '+file[:-2]+'('

# number of functions

In [8]:
for package, files in code.items():
    n = len(files.keys())
    print(f'{package:50} {n:3} functions')

C:\Users\micha\github\QDMlab\alignment_tools         9 functions
C:\Users\micha\github\QDMlab\analysis                4 functions
C:\Users\micha\github\QDMlab\Bz_and_UpCont           7 functions
C:\Users\micha\github\QDMlab\data_models             1 functions
C:\Users\micha\github\QDMlab\fitting                 9 functions
C:\Users\micha\github\QDMlab\io                     11 functions
C:\Users\micha\github\QDMlab\map_editing            14 functions
C:\Users\micha\github\QDMlab\net_moment_analysis     4 functions
C:\Users\micha\github\QDMlab\picking_tools           6 functions
C:\Users\micha\github\QDMlab\plotting               10 functions
C:\Users\micha\github\QDMlab\pressure_tools          1 functions
C:\Users\micha\github\QDMlab\utilities              24 functions
C:\Users\micha\github\QDMlab\validators              5 functions


In [9]:
docs = docstring.get_docstring(reload=True)
param_docs = {f:{} for f in docs.keys()}
n = 0
for f, doc in docs.items():
    param_idx = [i for i,v in enumerate(doc) if 'parameters' in v.lower()]
    
    if len(param_idx) != 1:
        print(f'{f:50}!!! NO PARAMETERS IN DOCSTRING')
        n += 1
    else:
        doc = doc[param_idx[0]+2:]
        colon_idx = [i for i,v in enumerate(doc) if ': ' in v if not 'Note:' in v if not 'See Also:' in v if not ' if ' in v]
        colon_idx.append(len(doc))
        
        for i, idx in enumerate(colon_idx[:-1]):
            d = doc[idx: colon_idx[i+1]]
            p = ''.join(d).split(':')[0][1:].lstrip()
            param_docs[f][p] = ''.join(d)

print(f'{n}/{len(docs.keys())} functions without parameter descriptions')

check_plot                                        !!! NO PARAMETERS IN DOCSTRING
estimate_coercivity                               !!! NO PARAMETERS IN DOCSTRING
updateRand                                        !!! NO PARAMETERS IN DOCSTRING
updatePlot                                        !!! NO PARAMETERS IN DOCSTRING
MITBxByFromBz                                     !!! NO PARAMETERS IN DOCSTRING
QDMBzFromBu                                       !!! NO PARAMETERS IN DOCSTRING
QuadBGsub                                         !!! NO PARAMETERS IN DOCSTRING
makeEvenArray                                     !!! NO PARAMETERS IN DOCSTRING
model_GPU                                         !!! NO PARAMETERS IN DOCSTRING
B111                                              !!! NO PARAMETERS IN DOCSTRING
fit_resonance                                     !!! NO PARAMETERS IN DOCSTRING
slope_correction                                  !!! NO PARAMETERS IN DOCSTRING
get_initial_guess           

# check where functions are called 

In [10]:
for package, files in code.items():
    print(package)
    print('='*50)    
    for file in files.keys():
        print('>>>>>', file)
        n = 0
        for _, files in code.items():
            for f, c in files.items():
                if f == file:
                    continue
                if f.startswith('test_'):
                    continue
                if call(file) in ''.join(c):
                    n += 1
                    print(f'     - {file} called in {f}')
                    idx = [i for i,l in enumerate(c) if call(file) in l]
                    for i in idx:
                        print(f'            {c[i].rstrip().lstrip()}')
        if n == 0:
            print(f'WARNING: {file} is never called')
        print('-'*50)
    print('='*50)    

C:\Users\micha\github\QDMlab\alignment_tools
>>>>> align_images.m
     - align_images.m called in get_transformed_maps.m
            [nTransForms, nRefFrames] = align_images(nFolders, 'transformFile', 0, ...
--------------------------------------------------
>>>>> get_image_tform.m
     - get_image_tform.m called in align_images.m
            [tForm, refFrame] = get_image_tform(movingLed, fixedLed, 'check', true);
            [tForm, refFrame] = get_image_tform(fixedLed, movingLed, 'check', true);
     - get_image_tform.m called in get_tform_multi.m
            [tForm, refFrame] = get_image_tform(movingLed, fixedLed, 'checkPlot', checkPlot);
            [tForm, refFrame] = get_image_tform(fixedLed, movingLed, 'checkPlot', checkPlot);
     - get_image_tform.m called in save_transformed_map.m
            [transForm, refFrame] = get_image_tform(movingData.(kwargs.reference), ...
     - get_image_tform.m called in subtract_blank.m
            [transForm, refFrame] = get_image_tform(fixedDa

# check where functions are called 

In [ ]:
for package, files in code.items():
    print(package)
    print('='*50)    
    for file in files.keys():
        print('>>>>>', file)
        n = 0
        for _, files in code.items():
            for f, c in files.items():
                if f == file:
                    continue
                if f.startswith('test_'):
                    continue
                if call(file) in ''.join(c):
                    n += 1
                    print(f'     - {file} called in {f}')
                    idx = [i for i,l in enumerate(c) if call(file) in l]
                    for i in idx:
                        print(f'            {c[i].rstrip().lstrip()}')
        if n == 0:
            print(f'WARNING: {file} is never called')
        print('-'*50)
    print('='*50)    

In [6]:
p = admin_tools.get_in_out_params()

In [12]:
p['fit_resonance']

{'parameters': ['expData', 'binSize', 'nRes', 'header'],
 'returns': ['fit'],
 'funcPath': 'C:\\Users\\micha\\github\\QDMlab\\fitting\\fit_resonance.m',
 'line': 'function fit = fit_resonance(expData, binSize, nRes, header, kwargs)\n',
 'indent': '',
 'kwargs': {'header': "'none'",
  'type': '2',
  'globalFraction': '0.5',
  'forceGuess': 'false',
  'checkPlot': 'false',
  'gaussianFit': 'false',
  'gaussianFilter': '0',
  'smoothDegree': '2',
  'diamond': "'doublet'",
  'slopeCorrection': 'false',
  'crop': "'none'",
  'fcrop': 'false'}}

In [18]:
admin_tools.arguments.first_line_comments(save=False, debug=False)

C:\Users\micha\github\QDMlab\fitting\B111.m
----------------------------------------------------------------------------------------------------
NO CHANGE
----------------------------------------------------------------------------------------------------

C:\Users\micha\github\QDMlab\Bz_and_UpCont\B111ToBz_CommLine.m
----------------------------------------------------------------------------------------------------
NO CHANGE
----------------------------------------------------------------------------------------------------

C:\Users\micha\github\QDMlab\utilities\BinImage.m
----------------------------------------------------------------------------------------------------
NO CHANGE
----------------------------------------------------------------------------------------------------

C:\Users\micha\github\QDMlab\map_editing\FitMoment.m
----------------------------------------------------------------------------------------------------
NO CHANGE
----------------------------------------